# Tabular Models

**SDV** supports modeling single table datasets. It provides unique features for making it easy
for the user to learn models and synthesize datasets. Some important features of `sdv.tabular` include:

* Support for tables with primary key
* Support to anonymize certain fields like addresses, emails, phone numbers, names and other
  PII information.
* Support for a number of different data types - categorical, numerical, discrete-ordinal
  and datetimes.
* Support multiple types of statistical and deep learning models:
  * GaussianCopula: A tool to model multivariate distributions using [copula functions](
    https://en.wikipedia.org/wiki/Copula_%28probability_theory%29). Based on our [Copulas Library](
    https://github.com/sdv-dev/Copulas).
  * CTGAN: A GAN-based Deep Learning data synthesizer that can generate synthetic tabular data with high 
    fidelity. Based on our [CTGAN Library](https://github.com/sdv-dev/CTGAN).

In this guide we will go through a series of steps that will let you discover the common
functionalities of the Tabular Models, including how to:

* Create an instance of a Tabular Model.
* Fit the instance to your data.
* Generate synthetic versions of your data.
* Use the a Tabular Model to anonymize PII information.
* Customize the data tranformations to improve the learning process.
* Specify inter-column constraints to improve the output quality.

<div class="alert alert-info">

**NOTE**

During this guide we will use the `GaussianCopula` model as an example but we
will be talking only about the functionalities that are shared across all the Tabular Models.
To discover specific features of the GaussianCopula in particular please visit the
[GaussianCopula User Guide](02_GaussianCopula_Model.ipynb).

</div>

## Load and explore tabular data

We will start by loading one of our demo datasets, the `student_placements`, which contains information
about MBA students that applied for placements during the year 2020.

In [1]:
# Setup logging and warnings - change ERROR to INFO for increased verbosity
import logging;
logging.basicConfig(level=logging.ERROR)

logging.getLogger().setLevel(level=logging.WARNING)
logging.getLogger('sdv').setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head().T

,0,1,2,3,4
student_id,155368,155369,155370,155371,155372
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0
employability_perc,55,86.5,75,66,96.8


As you can see, this table contains information about students which includes, among other things:

- Their id and gender
- Their grades and specializations
- Their work experience
- The salary that they where offered
- The duration and dates of their placement

You will notice that there is data with the following characteristics:

- There are float, integer, boolean, categorical and datetime values.
- There are some variables that have missing data. In particular, all the data related to the
  placement details is missing in the rows where the studen was not placed.

Let us use the `GaussianCopula` to learn this data and then sample synthetic data about new students
to see how well de model captures the characteristics indicated above.

In order to do this you will need to:

- Import the `sdv.tabular.GaussianCopula` class and create an instance of it.
- Call its `fit` method passing our table.
- Call its `sample` method indicating the number of synthetic rows that you want to generate.

In [3]:
from sdv.tabular import GaussianCopula

model = GaussianCopula()
model.fit(data)

<div class="alert alert-info">

**NOTE**

Notice that the model `fitting` process took care of transforming the different fields using the
appropriate [Reversible Data Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has
a format that the GaussianMultivariate model from the [copulas](https://github.com/sdv-dev/Copulas)
library can handle.

</div>

## Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from your model passing the number of rows that we want to generate.

In [4]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [5]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,155405,F,76.229732,94.593284,Commerce,79.883373,Sci&Tech,False,1,76.904263,Mkt&Fin,65.176872,23065.936713,True,2020-01-10 10:11:25.076688640,2020-10-09 20:23:24.138314240,3.0
1,155379,M,53.059233,63.862622,Commerce,63.642210,Comm&Mgmt,False,0,57.447956,Mkt&Fin,65.405370,NaN,False,NaT,NaT,12.0
2,155402,F,64.200159,63.187882,Commerce,64.146776,Sci&Tech,False,1,51.045344,Mkt&HR,70.021922,NaN,False,NaT,NaT,12.0
3,155484,M,63.904988,60.132955,Science,64.819941,Comm&Mgmt,False,0,69.446724,Mkt&HR,60.713842,NaN,False,NaT,NaT,12.0
4,155411,M,66.395611,83.085422,Commerce,66.367068,Comm&Mgmt,False,1,96.256997,Mkt&Fin,69.308773,26096.950286,True,2020-01-01 04:14:20.444153344,2021-01-14 18:12:15.921348608,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples` in the
`model.sample(<num_rows>)`. To test, try `model.sample(10000)`. Note that the original 
table only had ~200 rows.

</div>

## Save and Load

In many scenarios it will be convenient to generate synthetic versions of your data
directly in systems that do not have access to the original data source.

For example, if you may want to generate testing data on the fly inside a testing
environment that does not have access to your production database.

In these scenarios, fitting the model with real data every time that you need to generate
new data is feasible, so you will need to fit a model in your production environment,
save the fitted model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let's see how this process works.

### Save and share the model

Once you have fitted the model, all you need to do is call its `save` method passing the
name of the file in which you want to save the model. Note that the extension of the filename
is not relevant, but we will be using the `.pkl` extension to highlight that the serialization
protocol used is [pickle](https://docs.python.org/3/library/pickle.html).

In [6]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same directory in which you are
running SDV.

<div class="alert alert-info">

**IMPORTANT**
    
If you inspect the generated file you will notice that its size is much smaller
than the size of the data that you used to generate it. This is because the serialized model
contains **no information about the original data**, other than the parameters it needs to
generate synthetic versions of it. This means that you can safely share this `my_model.pkl`
file without the risc of disclosing any of your real data!
    
</div>

### Load the model and sample from it

The file you just generated can be send over to the system where the synthetic data will be
generated.

Once it is there, you can load it using the `GaussianCopula.load` method, and then you are
ready to sample new data from the loaded instance:

In [7]:
loaded = GaussianCopula.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">
    
**WARNING**
    
Notice that the system where the model is loaded needs to also have `sdv` installed,
otherwise it will not be able to load the model and use it.
    
</div>

## Advanced Usage

Now that we have discovered the basics, let's go over a few more advanced usage examples
and see the different arguments that we can pass to our Tabular Models in order to
customize them to our needs.

### Primary Key

One of the first things that you may have noticed when looking that demo data
is that there is a `student_id` column which acts as the primary key of the table,
and which is supposed to have unique values.

Indeed, if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [8]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe that there
are some values that appear more than once:

In [9]:
new_data.student_id.value_counts().max()

5

In [10]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
11,155453,M,69.654598,70.056574,Commerce,67.874446,Comm&Mgmt,False,1,86.284360,Mkt&HR,58.614099,24698.213688,True,2020-03-23 23:13:14.104106496,2020-06-20 05:19:03.296640768,3.0
41,155453,M,75.299661,73.869109,Science,71.342630,Sci&Tech,True,2,85.737509,Mkt&Fin,74.119876,33944.773050,True,2020-01-14 15:15:13.240806144,2020-08-14 17:58:12.691002880,3.0
88,155453,F,89.396785,91.861133,Science,80.516035,Sci&Tech,False,0,89.980046,Mkt&HR,72.486207,34728.899987,True,2020-01-31 13:49:43.710874368,2021-02-01 08:39:06.056437504,12.0
153,155453,M,49.541025,73.057570,Commerce,65.196605,Comm&Mgmt,False,0,82.196786,Mkt&Fin,57.843805,32756.943286,True,2020-01-21 13:59:21.772719104,2020-07-15 16:43:45.200062208,3.0
189,155453,M,79.684527,71.484912,Science,80.356433,Comm&Mgmt,False,1,90.580254,Mkt&Fin,58.599487,24394.504796,True,2020-01-20 19:07:18.568514816,2020-12-23 01:33:33.132940800,12.0


This happens because the model was not notified at any point about the fact that the
`student_id` had to be unique, so when it generates new data it will provoke collisions
sooner or later.

In order to solve this, we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [11]:
model = GaussianCopula(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)

As a result, the model will learn that this column must be unique and generate a unique
sequence of valures for the column:

In [12]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,67.139480,70.541369,Science,73.909414,Comm&Mgmt,False,1,79.732777,Mkt&HR,60.908340,32242.248647,True,2020-03-15 13:06:05.741990400,2021-01-03 20:04:29.762910720,12.0
1,1,F,72.530617,54.323909,Commerce,64.230747,Comm&Mgmt,False,0,93.220647,Mkt&HR,61.039847,28344.790181,True,2020-04-27 09:53:00.512752640,2021-01-15 14:06:37.296257024,3.0
2,2,M,58.182569,69.213492,Commerce,60.292438,Comm&Mgmt,False,1,57.865082,Mkt&Fin,62.828773,NaN,False,NaT,NaT,3.0
3,3,M,49.827961,50.644579,Commerce,56.767931,Sci&Tech,False,-1,58.559610,Mkt&HR,53.861864,NaN,False,NaT,NaT,6.0
4,4,M,55.559829,45.690447,Commerce,70.514807,Comm&Mgmt,False,1,51.729329,Mkt&Fin,57.233825,NaN,False,NaT,NaT,3.0


In [13]:
new_data.student_id.value_counts().max()

1

### Anonymize fields that contain Personally Identifiable Information (PII) 

There will be many cases where the data will contain Personally Identifiable Information
which we cannot disclose.

In these cases, we will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but does not contain
any of the original values.

Let's load a new dataset that contains a PII field, the `student_placements_pii` demo, and
try to generate synthetic versions of it that do not contain any of the PII fields.

<div class="alert alert-info">
    
**NOTE**
    
The `student_placements_pii` dataset is a modified version of the `student_placements`
dataset with one new field, `name`, which contains PII information about the students.
Notice that this additional `name` field has been simulated and does not correspond to data
from the real users.

</div>

In [14]:
data = load_tabular_demo('student_placements_pii')

In [15]:
data.head().T

,0,1,2,3,4
student_id,155368,155369,155370,155371,155372
name,Ryan Massey,Charles Frye,Corey Compton,John White III,Jeffery Robinson
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0


If we use our tabular model on this new data we will see how the synthetic
data that it generates discloses the names from the real students:

In [16]:
model = GaussianCopula(
    primary_key='student_id',
)
model.fit(data)

In [17]:
new_data = model.sample(200)
new_data.head()

,student_id,name,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,Patrick Johnson,M,69.377127,54.387866,Science,56.109282,Sci&Tech,False,0,65.973613,Mkt&HR,62.130353,NaN,False,NaT,NaT,6.0
1,1,Rebecca Crawford,M,62.141844,71.650294,Commerce,65.133683,Comm&Mgmt,False,1,64.286034,Mkt&Fin,61.377120,26295.254113,False,2020-02-03 07:47:13.260436224,2020-06-28 01:32:42.205131520,3.0
2,2,Amy Snow,M,55.228877,69.480341,Commerce,58.725051,Comm&Mgmt,False,2,96.144632,Mkt&HR,64.294505,28489.350858,True,2020-05-03 04:56:05.908792320,2021-01-03 11:55:20.664620544,3.0
3,3,Joshua Johnson,M,72.816545,77.782902,Science,60.544922,Sci&Tech,False,1,70.888556,Mkt&Fin,57.829595,24244.263920,True,2020-01-03 00:20:15.790958592,2020-11-29 16:58:03.153403392,12.0
4,4,Heather Zamora,M,57.530192,54.854550,Science,66.678126,Sci&Tech,False,0,96.511053,Mkt&HR,61.170059,NaN,False,NaT,NaT,12.0


In [18]:
new_data.name.isin(data.name).sum()

200

In order to solve this, we can pass an additional argument `anonymize_fields` to
our model when we create the instance.

This `anonymize_fields` argument will need to be a dictionary that contains:
- The name of the field that we want to anonymize.
- The category of the field that we want to use when we generate fake values for it.

The list complete list of possible categories can be seen in the [Faker Providers
](https://faker.readthedocs.io/en/master/providers.html) page, and it contains a huge
list of concepts such as:

- name
- address
- country
- city
- ssn
- credit_card_number
- credit_card_expier
- credit_card_security_code
- email
- telephone
- ...

In this case, since the field is an e-mail address, we will pass a dictionary indicating
the category `name`

In [19]:
model = GaussianCopula(
    primary_key='student_id',
    anonymize_fields={
        'name': 'name'
    }
)
model.fit(data)

As a result, we can see how the real `name` values have been replaced by other fake
names that were not taken from the real data that we learned.

In [20]:
new_data = model.sample(200)
new_data.head()

,student_id,name,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,Darius Calderon,M,59.546090,69.012324,Commerce,73.722700,Comm&Mgmt,False,1,86.960741,Mkt&HR,62.351579,26404.675479,True,2020-01-25 03:46:52.543413248,2020-10-26 21:32:25.171588608,12.0
1,1,Max Pham,M,80.753107,71.868977,Commerce,70.596648,Comm&Mgmt,False,0,67.986171,Mkt&Fin,58.014176,32785.804485,True,2020-01-01 00:04:49.353167872,2020-04-04 20:38:31.302868736,3.0
2,2,John Weber,M,49.977849,64.724303,Commerce,50.153330,Comm&Mgmt,False,1,67.282311,Mkt&HR,54.670757,NaN,False,NaT,NaT,12.0
3,3,Johnny Jackson,M,76.309337,54.004954,Science,63.042458,Sci&Tech,False,0,85.795031,Mkt&HR,62.129727,33565.415910,False,2020-01-02 18:28:58.228424704,2020-09-10 08:23:14.184352512,12.0
4,4,Juan Vasquez,M,54.134037,78.132432,Commerce,58.479991,Comm&Mgmt,False,0,94.586020,Mkt&Fin,60.311280,31813.585314,True,2020-01-03 01:18:55.917426688,2020-07-25 16:33:18.059055360,12.0


In [21]:
new_data.name.isin(data.name).sum()

2

### Use different transformers

One thing that you may have noticed when executing the previous steps is that the fitting
process took much longer on the `student_placements_pii` dataset than it took on the
previous version that did not contain the sudent `name`s.

This happens because the `name` field is interpreted as a categorical variable, which the
Tabular Model [one-hot encoded](https://en.wikipedia.org/wiki/One-hot) generating 215 new
columns that it had to learn afterwards.

This transformation, which in this case was very inefficient, happens because the Tabular
Models apply [Reversible Data Transforms](https://github.com/sdv-dev/RDT) under the hood
to transform all the non-numerical variables, which the underlying models cannot handle,
into numerical representations which they can properly work with.

In the case of the `GaussianCopula`, the default transformation is a One-Hot encoding,
which can work very well with variables that have a little number of different values,
but which is very inefficient in cases where there is a large number of values.

For this reason, the Tabular Models have an additional argument called `field_transformers`
that let you select which transformer to apply to each column. This `field_transformers`
argument must be passed as a `dict` which contains the name of the fields for which
we want to use a transformer different than the default, and the name of the transformer
that we want to use.

Possible transformer names are:

* `integer`: Uses a `NumericalTransformer` of dtype `int`.
* `float`: Uses a `NumericalTransformer` of dtype `float`.
* `categorical`: Uses a `CategoricalTransformer` without gaussian noise.
* `categorical_fuzzy`: Uses a `CategoricalTransformer` adding gaussian noise.
* `one_hot_encoding`: Uses a `OneHotEncodingTransformer`.
* `label_encoding`: Uses a `LabelEncodingTransformer`.
* `boolean`: Uses a `BooleanTransformer`.
* `datetime`: Uses a `DatetimeTransformer`.

**NOTE**: For additional details about each one of the transformers, please visit [RDT](
https://github.com/sdv-dev/RDT)

Let's now try to improve the previous fitting process by changing the transformer
that we use for the `name` field to something other than the default.

As an example, we will use the `label_encoding` transformer, which instead of generating
one column for each possible value, it just replaces each value with a unique integer value.

In [22]:
model = GaussianCopula(
    primary_key='student_id',
    anonymize_fields={
        'name': 'name'
    },
    field_transformers={
        'name': 'label_encoding'
    }
)
model.fit(data)

In [23]:
new_data = model.sample(200)
new_data.head()

,student_id,name,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,Brenda Finley,F,80.755701,64.211528,Commerce,69.686663,Comm&Mgmt,False,1,63.035237,Mkt&HR,64.514951,28085.393371,True,2020-02-14 22:15:08.656968960,2020-09-23 13:25:17.775665408,3.0
1,1,Aaron Johnson,F,62.620598,57.979099,Commerce,55.903551,Sci&Tech,True,-1,50.069491,Mkt&HR,53.693381,20331.129712,True,2020-06-30 18:27:43.709538048,2020-11-06 03:31:15.367439616,6.0
2,2,Madeline Bell,M,67.762698,69.547603,Science,57.798505,Sci&Tech,False,1,60.396142,Mkt&HR,58.918337,25769.670733,True,2021-02-10 16:59:55.419933696,2020-05-26 02:34:52.481650944,3.0
3,3,Laura Tyler,M,75.860493,65.088318,Commerce,70.944755,Comm&Mgmt,False,1,68.365267,Mkt&HR,62.197199,28726.227473,True,2020-01-16 02:37:40.392923392,2020-08-20 12:39:24.729924352,3.0
4,4,Courtney Christensen,M,69.753001,55.807033,Commerce,66.063034,Comm&Mgmt,False,1,61.095079,Mkt&Fin,55.729902,24761.537458,True,2020-01-06 23:30:01.212045312,2020-08-26 00:24:12.258295040,3.0
